tensorflow 공식 튜토리얼 generative models DCGAN  : 
    https://github.com/tensorflow/tensorflow/blob/r1.11/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb

##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License").

# tf.keras 와 eager로 구현한 DCGAN 예제

<table class="tfo-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

 [tf.keras](https://www.tensorflow.org/programmers_guide/keras) 와 [eager execution](https://www.tensorflow.org/programmers_guide/eager)을 사용하여 손글씨 숫자 이미지를 생성하는 예제를 설명하려 합니다.

많은 방법이 있지만 이 글에서는 Deep Convolutional Generative Adverserial Networks([DCGAN](https://arxiv.org/pdf/1511.06434.pdf))를 사용하도록 하겠습니다.

이 DCGAN 모델은 Colab(Tesla K80 1개)에서 학습할 때 에폭당 30초 정도 시간이 걸립니다.

 (using tf.contrib.eager.defun to create graph functions)

아래 그림은 150 에폭 동안 모델을 학습한 후 결과입니다.

![sample output](https://tensorflow.org/images/gan/dcgan.gif)

In [ ]:
#  gifs 형태로 생성하기 위해 imageio를 설치합니다.
!pip install imageio

## Import TensorFlow and enable eager execution

In [ ]:
from __future__ import absolute_import, division, print_function

# TensorFlow 버전1.10 이상을 Import하고 and eager execution enable선언합니다.
import tensorflow as tf
tf.enable_eager_execution()

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
from IPython import display

## 데이터셋을 불러옵니다.

우리는 MNIST 데이터 셋을 사용합니다. MNIST는 손글씨 숫자 데이터셋입니다.
생성 모델 DCGAN은 손글씨 숫자를 생성할 것 입니다.


In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
#  [-1, 1] 범위로 이미지를 정규화(normalizing)합니다.
train_images = (train_images - 127.5) / 127.5

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

## tf.data를 사용하여 데이터셋을 섞어주고(shuffle) 미니배치로 만들어줍니다.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## generator 모델과 discriminator 모델을 만들어줍니다.

* **Generator** 
  * Generator 모델은 **discriminator를 속이기 위해 그에 알맞은 이미지를 만들어내려 합니다.**.
  *  Conv2DTranspose (Upsampling) layer로 구성되어 있습니다. fully connected layer로 시작해서 원하는 사이즈가 될 때 까지[여기서는 (28, 28, 1)] 이미지를 2배로 늘려주는 unsampling 작업을 해줍니다. 
  *  **leaky relu** 활성화 함수를 사용합니다. 마지막 layer만 tanh 활성화 함수를 사용합니다. 
  
* **Discriminator**
  * **The discriminator 모델은 가짜 이미지와 진짜 이미지를 구별하려 합니다.**
  * 다시 말해, discriminator가 하는일은 "generator가 생성한 이미지"와 "실제 MNIST 이미지"를 분류하는 일입니다.
  * **generator는 discriminator를 속일 만큼 충분한 성능이 되도록(discriminator가 생성된 이미지를 진짜라고 여기도록) 설계되어야 합니다. **.

In [ ]:
class Generator(tf.keras.Model):
  def __init__(self):
    super(Generator, self).__init__()
    self.fc1 = tf.keras.layers.Dense(7*7*64, use_bias=False)
    self.batchnorm1 = tf.keras.layers.BatchNormalization()
    
    self.conv1 = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False)
    self.batchnorm2 = tf.keras.layers.BatchNormalization()
    
    self.conv2 = tf.keras.layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False)
    self.batchnorm3 = tf.keras.layers.BatchNormalization()
    
    self.conv3 = tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False)

  def call(self, x, training=True):
    x = self.fc1(x)
    x = self.batchnorm1(x, training=training)
    x = tf.nn.relu(x)

    x = tf.reshape(x, shape=(-1, 7, 7, 64))

    x = self.conv1(x)
    x = self.batchnorm2(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2(x)
    x = self.batchnorm3(x, training=training)
    x = tf.nn.relu(x)

    x = tf.nn.tanh(self.conv3(x))  
    return x

In [ ]:
class Discriminator(tf.keras.Model):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')
    self.conv2 = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')
    self.dropout = tf.keras.layers.Dropout(0.3)
    self.flatten = tf.keras.layers.Flatten()
    self.fc1 = tf.keras.layers.Dense(1)

  def call(self, x, training=True):
    x = tf.nn.leaky_relu(self.conv1(x))
    x = self.dropout(x, training=training)
    x = tf.nn.leaky_relu(self.conv2(x))
    x = self.dropout(x, training=training)
    x = self.flatten(x)
    x = self.fc1(x)
    return x

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
# Defun은  gives 10 secs/epoch performance boost
generator.call = tf.contrib.eager.defun(generator.call)
discriminator.call = tf.contrib.eager.defun(discriminator.call)

## loss 함수와 옵티마이저를 정의합니다.

* **Discriminator loss**
  * discriminator loss 함수는  2 종류의 입력값을 받습니다. ; **실제 이미지, 생성된 이미지**
  * real_loss 는 **실제 이미지**의 sigmoid cross entropy loss 입니다. 원하는 값(target)이 1이 되도록 합니다.
  * generated_loss는  **생성된 이미지**의 sigmoid cross entropy loss 입니다. 원하는 값(target)이 0이 되도록 합니다.
  * total_loss 는 위 두 loss의 합 입니다. 
  
* **Generator loss**
  * generated_loss는 생성된 이미지의 sigmoid cross entropy loss 입니다. 원하는 값(target)이 1이 되도록 합니다.
  

* discriminator와 the generator 옵티마이저들은 분리하여 학습합니다.

In [ ]:
def discriminator_loss(real_output, generated_output):
    # [1,1,...,1] 실제 이미지는 1으로 두기로 합니다.
    # our generated examples to look like it
    real_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.ones_like(real_output), logits=real_output)

    # [0,0,...,0] 생성된 이미지는 fake이기 때문에 0으로 두기로 합니다.
    generated_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.zeros_like(generated_output), logits=generated_output)

    total_loss = real_loss + generated_loss

    return total_loss

In [ ]:
def generator_loss(generated_output):
    return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output), generated_output)

In [ ]:
discriminator_optimizer = tf.train.AdamOptimizer(1e-4)
generator_optimizer = tf.train.AdamOptimizer(1e-4)

## Checkpoints

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## 학습하기

* 주어진 데이터셋를 반복하여(iterating) 학습합니다.
* generator는 입력값으로 일종의 **noise** 를 받습니다. generator 모델은 손글씨 숫자로 보이는 이미지를 출력할 것입니다.
* discriminator는 **generator에서 생성된 이미지**와 **실제 MNIST 이미지**를 입력값으로 받습니다.
* 그 다음, generator loss와 discriminator loss를 계산합니다. 
* 그리고나서, generator와 the discriminator의 변수들 (입력값들) 양쪽에 대해서 loss의 그레디언트(gradients)를 계산합니다. 이 결과들을 옵티마이저에 적용합니다.

## 이미지를 생성하기

* 학습 후, 이미지를 생성해봅시다! 
* 입력값인 noise 어레이를 만들어서 generator에 넣습니다.
* generator은 noise를 손글씨 이미지로 변환해줍니다!
* Last step is to plot the predictions and **voila!**

In [ ]:
EPOCHS = 150
noise_dim = 100
num_examples_to_generate = 16

# 생성에 사용된 noise를 상수로 저장해서 나중에 불러올 수 있도록 합니다.
# gan모델의 성능이 향상되는 과정을 더 쉽게 보여주도록 해줍니다.
random_vector_for_generation = tf.random_normal([num_examples_to_generate,
                                                 noise_dim])

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # make sure the training parameter is set to False because we
  # don't want to train the batchnorm layer when doing inference.
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
        
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
def train(dataset, epochs, noise_dim):  
  for epoch in range(epochs):
    start = time.time()
    
    for images in dataset:
      # 균등 분포로 샘플링하여 noise를 생성하기
      noise = tf.random_normal([BATCH_SIZE, noise_dim])
      
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
      
        real_output = discriminator(images, training=True)
        generated_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(generated_output)
        disc_loss = discriminator_loss(real_output, generated_output)
        
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.variables)
      
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.variables))

      
    if epoch % 1 == 0:
      display.clear_output(wait=True)
      generate_and_save_images(generator,
                               epoch + 1,
                               random_vector_for_generation)
    
    # 15 에폭마다 checkpoint를 저장
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print ('Time taken for epoch {} is {} sec'.format(epoch + 1,
                                                      time.time()-start))
  # 마지막 에폭을 돌고난 후 생성하기 
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           random_vector_for_generation)

In [ ]:
train(train_dataset, EPOCHS, noise_dim)

## 가장 최근 checkpoint를 저장하기

In [ ]:
# checkpoint_dir 안에 저장 
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## 에폭 수에 따른 이미지 보여주기

In [ ]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

## 저장된 이미지를 GIF 로 생성하기

<!-- TODO(markdaoust): Remove the hack when Ipython version is updated -->


In [ ]:
with imageio.get_writer('dcgan.gif', mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# jupyter notebook에서 gif를 보여주기 위한 코드입니다. 
os.system('cp dcgan.gif dcgan.gif.png')

In [ ]:
display.Image(filename="dcgan.gif.png")

생성된 이미지 애니메이션를 다운로드 하기 위한 Colab 코드(아래):

In [ ]:
#from google.colab import files
#files.download('dcgan.gif')

이 글은 2018 컨트리뷰톤에서 Contribute to Keras 프로젝트로 진행했습니다.번역자: 김형섭 이메일: times21c@gmail.com